In [1]:
tokenized_comments_fname = '/mnt/lovit/works/fastcampus_text_ml/3rd//data/comments_172movies/merged_comments_tokenized.txt'
with open(tokenized_comments_fname, encoding='utf-8') as f:
    for _ in range(3):
        print(next(f).strip())

72523	명불허전	1
72523	왠지 고사 피의 중간 고사 보다 재미 가 없을 듯해요 만약 보게 된다 면 실망 할듯	1
72523	티아라 사랑 해 ㅜ	10


In [2]:
def load_reviews(fname, n_limit=-1):
    with open(fname, encoding='utf-8') as f:
        idxs = []
        scores = []
        texts = []
        
        for i, doc in enumerate(f):
            if n_limit > 0 and i >= n_limit:
                break
                
            try:
                idx, text, score = doc.strip().split('\t')
                idxs.append(idx)
                texts.append(text)
                scores.append(int(score))
                
            except Exception as e:
                print(e)
                continue
                
    return idxs, texts, scores

In [3]:
idxs, texts, scores = load_reviews(tokenized_comments_fname)
print(len(scores))

3280685


In [4]:
from collections import Counter
word_counter = Counter([word for text in texts for word in text.split()])

In [5]:
sorted(word_counter.items(), key=lambda x:x[1], reverse=True)[:20]

[('영화', 1412516),
 ('이', 764006),
 ('관람객', 585858),
 ('는', 459876),
 ('가', 438771),
 ('도', 418073),
 ('의', 403943),
 ('다', 381746),
 ('재밌', 370724),
 ('재미', 344634),
 ('너무', 335529),
 ('ㅋㅋ', 321284),
 ('정말', 297962),
 ('고', 294644),
 ('을', 270826),
 ('에', 266720),
 ('한', 263385),
 ('를', 263311),
 ('연기', 255673),
 ('최고', 254291)]

In [6]:
n_before = len(word_counter)

min_count = 10
word_dictionary = {
    word:freq for word,freq in word_counter.items()
    if freq >= min_count and len(word) > 1
}

n_after  = len(word_dictionary)

print('%d --> %d' % (n_before, n_after))

341744 --> 49901


In [7]:
for score, freq in sorted(Counter(scores).items()):
    perc = 100 * freq / len(scores)
    print('score = %d: (%d, %.3f perc)' % (score, freq, perc))

score = 1: (320898, 9.781 perc)
score = 2: (34351, 1.047 perc)
score = 3: (35580, 1.085 perc)
score = 4: (39742, 1.211 perc)
score = 5: (78250, 2.385 perc)
score = 6: (95834, 2.921 perc)
score = 7: (149618, 4.561 perc)
score = 8: (268622, 8.188 perc)
score = 9: (344905, 10.513 perc)
score = 10: (1912885, 58.307 perc)


In [8]:
import numpy as np

train_texts = []
train_label = []

for text, score in zip(texts, scores):

    # skip 4 ~ 8 scored reviews
    if 4 <= score <= 8:
        continue

    # skip empty reviews
    words = [word for word in text.split() if word in word_dictionary]
    if not words:
        continue

    # append text and label
    train_texts.append(words)
    train_label.append(1 if score > 8 else 0)

train_label = np.asarray(train_label)

print('train data: %d --> %d' % (len(texts), len(train_texts)))

for label, freq in Counter(train_label).items():
    perc = 100 * freq / len(train_label)
    print('label = %d: (%d, %.3f perc)' % (label, freq, perc))

train data: 3280685 --> 2622411
label = 0: (387039, 14.759 perc)
label = 1: (2235372, 85.241 perc)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x:x, lowercase=False)
train_x = vectorizer.fit_transform(train_texts)
train_x.shape

(2622411, 49896)

In [18]:
# with open('./tmp/sentiment_vocablist', 'w', encoding='utf-8') as f:
#     for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]):
#         f.write('%s\n' % word)
vocablist = [
    word for word, _ in sorted(
        vectorizer.vocabulary_.items(), key=lambda x:x[1]
    )
]

vocabcount = [word_dictionary[word] for word in vocablist]

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

logistic_l2 = LogisticRegression(verbose=True)
logistic_l2.fit(train_x, train_label)
predict_label_l2 = logistic_l2.predict(train_x)

logistic_l1 = LogisticRegression(verbose=True, penalty='l1')
logistic_l1.fit(train_x, train_label)
predict_label_l1 = logistic_l2.predict(train_x)

bayes = BernoulliNB()
bayes.fit(train_x, train_label)
predict_label_nb = bayes.predict(train_x)

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[LibLinear][LibLinear]

In [12]:
import numpy as np

X = train_x
y = train_label


unique = np.unique(y)
print(unique)

[0 1]


In [13]:
n_features = X.shape[1]
n_classes = unique.shape[0]
coefs = np.zeros((n_classes, n_features))

Py = np.asarray(X.sum(axis=0)).reshape(-1)
Py = Py / Py.sum()
Px = np.zeros(n_classes)
Pxy = np.zeros((n_classes, n_features))
Py_x = np.zeros((n_classes, n_features))

for c, label in enumerate(unique):
    indices = np.where(y == label)[0]
    Py_x_ = np.asarray(X[indices].sum(axis=0)).reshape(-1)
    Py_x_ = Py_x_ / Py_x_.sum()
    Pxy[c] = (Py_x_ / Py)
    Py_x[c] = Py_x_
    Px[c] = indices.shape[0]
Px = Px / Px.sum()
Px = np.log(Px)

PMIxy = np.log(Pxy)
PMIxy = np.nan_to_num(PMIxy)
PMIxy[np.where(PMIxy < -5)] = 0
coefs = (PMIxy[1] - PMIxy[0])
t = -(Px[1] - Px[0])

keyword_score = Py_x[1] - Py_x[0]

def predict(X, coefs, t=0):
    y = X.dot(coefs)
    y[np.where(y >= t)[0]]= 1
    y[np.where(y < t)[0]] = 0
    return y

def accuracy(answer, prediction):
    return (answer == prediction).sum() / answer.shape[0]

predict_label = predict(X, coefs, t)
print('l2 = {}'.format(accuracy(train_label, predict_label_l2)))
print('l1 = {}'.format(accuracy(train_label, predict_label_l1)))
print('NB = {}'.format(accuracy(train_label, predict_label_nb)))
print('pmi (t={:.3f}) = {}'.format(t, accuracy(train_label, predict_label)))

l2 = 0.9331332884128384
l1 = 0.9331332884128384
NB = 0.9031345582366761
pmi (t=-1.754) = 0.9067373497136795


/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log


In [36]:
# softmax version

n_features = X.shape[1]
n_classes = unique.shape[0]
coefs = np.zeros((n_classes, n_features))

Py = np.asarray(X.sum(axis=0)).reshape(-1)
Py = Py / Py.sum()
Px = np.zeros(n_classes)
Pxy = np.zeros((n_classes, n_features))
Py_x = np.zeros((n_classes, n_features))

for c, label in enumerate(unique):
    indices = np.where(y == label)[0]
    Py_x_ = np.asarray(X[indices].sum(axis=0)).reshape(-1)
    Py_x_ = Py_x_ / Py_x_.sum()
    Pxy[c] = (Py_x_ / Py)
    Py_x[c] = Py_x_
    Px[c] = indices.shape[0]
Px = Px / Px.sum()
Px = np.log(Px)

PMIxy = np.log(Pxy)
PMIxy = np.nan_to_num(PMIxy)
PMIxy[np.where(PMIxy < -5)] = 0
coefs = PMIxy.copy().T
beta = Px

def softmax(X, coefs, beta):
    prod = X.dot(coefs)
    prod += beta
    y = prod.argmax(axis=1)
    return y

def softmax_prob(X, coefs, beta):
    prod = X.dot(coefs)
    prod += beta
    prob = np.exp(prod)
    prob /= prob.sum(axis=1)[:,np.newaxis]
    return prob

def accuracy(answer, prediction):
    return (answer == prediction).sum() / answer.shape[0]

predict_label = softmax(X, coefs, beta)
print('l2 = {}'.format(accuracy(train_label, predict_label_l2)))
print('l1 = {}'.format(accuracy(train_label, predict_label_l1)))
print('NB = {}'.format(accuracy(train_label, predict_label_nb)))
print('pmi (t={:.3f}) = {}'.format(t, accuracy(train_label, predict_label)))

/home/lovit/anaconda2/envs/scrapper/lib/python3.6/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


l2 = 0.9331332884128384
l1 = 0.9331332884128384
NB = 0.9031345582366761
pmi (t=-1.754) = 0.9067373497136795


In [15]:
beta

array([-1.91332394, -0.15968647])

In [16]:
words = [vocablist[idx] for idx in keyword_score.argsort()[::-1][:200]]
print('## positive term ##')
for i in range(20):
    print('| {} |'.format(' | '.join(words[i*10:(i+1)*10])))

## positive term ##
| 관람객 | 재밌 | 최고 | 습니다 | 정말 | 감동 | 연기 | ㅠㅠ | 대박 | 어요 |
| 보고 | ㅎㅎ | 입니다 | 진짜 | 재밋 | 완전 | 합니다 | 기대 | 었어요 | 좋았 |
| 네요 | 강추 | 있었 | 눈물 | 봤어요 | 너무 | 생각 | 있는 | 적이 | 봐야 |
| 다시 | 역시 | 있게 | 재미 | 추천 | 가슴 | 있고 | 한번 | 필요 | ㅋㅋ |
| 사랑 | 마지막 | 영화 | 잼있 | 우리 | 오랜만에 | 감사 | 있어 | 멋있 | 봐도 |
| 마음 | 울었 | 꿀잼 | 좋아 | 액션 | 긴장 | 같아요 | 말이 | 에요 | 였습니다 |
| ㅜㅜ | 적인 | 스릴 | 봤는데 | 여운이 | 괜찮 | 슬프 | 좋고 | 몰입 | 좋은 |
| 분들 | 보세요 | 모두 | 소름 | 하게 | 간만에 | 후회 | 짱짱 | 봤네요 | 명작 |
| 잘봤 | 굿굿 | 였어요 | 하정우 | 함께 | 두번 | 었음 | 들의 | 따뜻 | 까지 |
| 아깝지 | 가족 | 인생 | 멋진 | 잘하 | 만큼 | 엄청 | 시간가는줄 | 매력 | 조금 |
| 탄탄 | 볼만 | 안하고 | 만점 | 약간 | 올해 | 계속 | 음악 | 펑펑 | 지금 |
| 꼭보세요 | 반전 | 아주 | 웃고 | 재밋어요 | 유아인 | 오늘 | 모든 | 보게 | 해요 |
| 있다 | 가장 | 하지만 | 싶다 | 울고 | 흥미 | 없이 | 또보고싶 | 작품 | 였다 |
| 멋지 | 짱이 | 빨리 | 있네요 | 모습 | 먹먹 | 많은 | 즐거 | 않았 | 행복 |
| 훌륭 | 황정민 | 마블 | 지만 | 화이팅 | 대단 | 쵝오 | 특히 | 있음 | 었다 |
| 빠져 | 완벽 | 그리고 | 눈을 | 충분 | 표현 | 만족 | 아쉬 | 않은 | 잔잔 |
| 원빈 | 예요 | 아버지 | 아름다운 | 됩니 | 웃다가 | 잊지 | 신선 | 볼수 | go |
| 싶은 | 있어서 | 코믹 | 나름 | 여운 | 슬픈 | 부모님 | 봤음 | 놀란 | 했지만 |
|

In [17]:
words = [vocablist[idx] for idx in keyword_score.argsort()[:200]]
print('## negative term ##')
for i in range(20):
    print('| {} |'.format(' | '.join(words[i*10:(i+1)*10])))

## negative term ##
| 평점 | 알바 | 쓰레기 | 1점 | 없고 | 지루 | 별로 | ㅡㅡ | 스토리 | 아니 |
| 실망 | 하고 | 아깝다 | 억지 | 없는 | 없다 | 그냥 | 내용 | 이게 | 선동 |
| 최악 | 감독 | 없음 | 이런 | 이건 | OO | 최악의 | 수준 | 아까 | 솔직 |
| 유치 | 만들 | 라고 | 짜증 | 노잼 | 없어 | 아까운 | 0점 | 으로 | 무슨 |
| 이거 | 개연성 | 돈아까 | 하는 | 정도 | 차라리 | 시간 | 보다가 | 것도 | 하나 |
| 뻔한 | 사람 | 아깝 | ㅉㅉ | 나오 | 같은 | 려고 | 때문에 | 건지 | 하다 |
| 해서 | 전개 | 광주 | 렇게 | 효주 | 떨어 | 아닌 | 겠다 | 돈아깝 | 이딴 |
| 중간 | 했다 | 점수 | 절대 | 이걸 | 관객 | 안되 | 돈주고 | 인지 | 폭동 |
| 7점 | 드라마 | 도대체 | 졸작 | 네이버 | 거지 | 보지마 | 높아 | 왜곡 | 보지마세요 |
| 설정 | 만든 | 그만 | 폭동이 | CG | 비추 | 적당 | 높은 | 장난 | 준다 |
| 놓고 | 그나마 | 엉성 | 내가 | 없네 | 인가 | 제발 | 들은 | 이하 | 제작 |
| 없어서 | 돈이 | 연출 | 어이 | 시나리오 | 건가 | 별루 | 미화 | 느낌 | 캐스팅 |
| 여자 | 지도 | 그닥 | 초딩 | 짜리 | 잖아 | 북한 | 뻔하 | 2점 | 주인공 |
| 낭비 | 내돈 | 아님 | 18 | 허접 | 못하 | 6점 | 하기 | 라도 | 8점 |
| 감성팔이 | 디워 | 답답 | 광고 | 보지 | 그래 | ㅅㅂ | 높다 | 아무리 | 3류 |
| cg | 밖에 | 5점 | 미국 | 3점 | 일본 | O기 | 마라 | 어이없 | 예고편 |
| 엉망 | 신파 | 댓글 | 이나 | 없었 | 조작 | 알았 | 들이 | 막장 | 주고 |
| 애들 | 그저 | 삼류 | 뭐가 | 망작 | 진심 | 뭐냐 | 존나 | 허무 | 뭐야 |
| 왜이 | 인데 | 떻